# Predicting 30-day Hospital Readmission Risk (Neural Network)
**Dataset:** Simulated realistic hospital CSV (generated within this notebook)  
**Model:** Multi-Layer Perceptron (MLPClassifier) with hyperparameter tuning  
**Notebook contents:** Data generation → Full EDA → Preprocessing → Modeling → Evaluation → Saving artifacts

*Author: austine makwka*  
*Generated on: 2025-11-07*


In [ ]:
# Imports and configuration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report
import joblib
import os
plt.rcParams['figure.figsize'] = (8,5)
np.random.seed(42)


In [ ]:
# Generate a realistic simulated hospital dataset and save as CSV
def generate_hospital_dataset(n=5000, seed=42):
    rng = np.random.RandomState(seed)
    patient_id = np.arange(100000, 100000 + n)
    age = rng.randint(18, 90, size=n)
    gender = rng.choice(['Male', 'Female'], size=n, p=[0.48, 0.52])
    diagnoses = ['Heart Failure', 'Pneumonia', 'COPD', 'Diabetes', 'Sepsis', 'Kidney Disease', 'Stroke', 'Orthopedic']
    diagnosis = rng.choice(diagnoses, size=n, p=[0.15,0.12,0.12,0.18,0.08,0.10,0.10,0.15])
    num_lab_procedures = rng.poisson(10, size=n) + 1
    num_medications = rng.poisson(5, size=n) + 1
    time_in_hospital = np.clip(rng.poisson(4, size=n) + 1, 1, 30)
    discharge_disposition = rng.choice(['Home', 'SNF', 'Rehab', 'Against Medical Advice', 'Expired'], size=n, p=[0.7,0.12,0.08,0.08,0.02])
    # Simulate some comorbidity features
    charlson_score = np.clip(rng.poisson(2, size=n) + (age/50).astype(int), 0, 15)
    prior_admissions = rng.poisson(1.2, size=n)
    # Social factors
    insurance = rng.choice(['Medicare', 'Medicaid', 'Private', 'Uninsured'], size=n, p=[0.4,0.2,0.35,0.05])
    # Base risk for readmission
    base = 0.03 + 0.02*(prior_admissions) + 0.02*(charlson_score/10) + 0.01*(time_in_hospital/5)
    # Diagnosis-specific risk adjustments
    diag_risk = {'Heart Failure':0.08, 'Pneumonia':0.05, 'COPD':0.06, 'Diabetes':0.03, 'Sepsis':0.07, 'Kidney Disease':0.06, 'Stroke':0.04, 'Orthopedic':0.02}
    diag_adj = np.array([diag_risk[d] for d in diagnosis])
    readmit_prob = np.clip(base + diag_adj + rng.normal(0, 0.03, size=n), 0, 1)
    readmitted = rng.binomial(1, readmit_prob)
    df = pd.DataFrame({
        'patient_id': patient_id,
        'age': age,
        'gender': gender,
        'diagnosis': diagnosis,
        'num_lab_procedures': num_lab_procedures,
        'num_medications': num_medications,
        'time_in_hospital': time_in_hospital,
        'discharge_disposition': discharge_disposition,
        'charlson_score': charlson_score,
        'prior_admissions': prior_admissions,
        'insurance': insurance,
        'readmitted_30d': readmitted
    })
    # Introduce missingness
    for col in ['charlson_score', 'insurance']:
        idx = rng.choice(n, size=int(0.03*n), replace=False)
        df.loc[idx, col] = np.nan
    return df

df = generate_hospital_dataset(n=4000)
df.to_csv('hospital_data.csv', index=False)
print('Saved hospital_data.csv with shape', df.shape)
df.head()


In [ ]:
# Exploratory Data Analysis (EDA) - Distributions and counts
df = pd.read_csv('hospital_data.csv')
print('Dataset shape:', df.shape)
print('\nReadmission rate (overall):', df['readmitted_30d'].mean())

# Age distribution
plt.hist(df['age'].dropna(), bins=20)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

# Time in hospital distribution
plt.hist(df['time_in_hospital'], bins=15)
plt.title('Length of Stay Distribution')
plt.xlabel('Days')
plt.ylabel('Count')
plt.show()

# Number of medications distribution
plt.hist(df['num_medications'], bins=15)
plt.title('Number of Medications Distribution')
plt.xlabel('Num medications')
plt.ylabel('Count')
plt.show()

# Diagnosis frequency
diag_counts = df['diagnosis'].value_counts()
diag_counts.plot(kind='bar')
plt.title('Diagnosis Frequency')
plt.xlabel('Diagnosis')
plt.ylabel('Count')
plt.show()

# Readmission rate by diagnosis
readmit_by_diag = df.groupby('diagnosis')['readmitted_30d'].mean().sort_values(ascending=False)
readmit_by_diag.plot(kind='bar')
plt.title('Readmission Rate by Diagnosis')
plt.ylabel('Readmission Rate')
plt.show()


In [ ]:
# Correlation heatmap (numerical features) using matplotlib only
num_cols = ['age','num_lab_procedures','num_medications','time_in_hospital','charlson_score','prior_admissions','readmitted_30d']
corr = df[num_cols].corr()
print(corr)

fig, ax = plt.subplots(figsize=(8,6))
cax = ax.matshow(corr, vmin=-1, vmax=1)
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha='left')
plt.yticks(range(len(num_cols)), num_cols)
fig.colorbar(cax)
plt.title('Correlation Matrix (numerical features)', pad=20)
plt.show()


In [ ]:
# Preprocessing: define pipelines for numerical and categorical features
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

target = 'readmitted_30d'
X = df.drop(columns=[target, 'patient_id'])
y = df[target]

numeric_features = ['age','num_lab_procedures','num_medications','time_in_hospital','charlson_score','prior_admissions']
categorical_features = ['gender','diagnosis','discharge_disposition','insurance']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Run a quick transform to see feature shape
X_prep = preprocessor.fit_transform(X)
print('Transformed feature matrix shape:', X_prep.shape)


In [ ]:
# Train/Validation/Test split (70/15/15 stratified)
X_full = X
y_full = y
X_temp, X_test, y_temp, y_test = train_test_split(X_full, y_full, test_size=0.15, stratify=y_full, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=42)
print('Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)

# Build pipeline with MLPClassifier
pipeline = Pipeline([
    ('pre', preprocessor),
    ('mlp', MLPClassifier(max_iter=300, random_state=42))
])

param_grid = {
    'mlp__hidden_layer_sizes': [(32,), (64,), (64,32)],
    'mlp__alpha': [1e-4, 1e-3],
    'mlp__learning_rate_init': [1e-3, 5e-4]
}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=cv, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
print('Best params:', grid.best_params_)
best_model = grid.best_estimator_


In [ ]:
# Evaluation on validation and test sets
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Validation
y_val_proba = best_model.predict_proba(X_val)[:,1]
y_val_pred = best_model.predict(X_val)
print('Validation AUC:', roc_auc_score(y_val, y_val_proba))
print('Validation Precision:', precision_score(y_val, y_val_pred))
print('Validation Recall:', recall_score(y_val, y_val_pred))
print('Validation F1:', f1_score(y_val, y_val_pred))
print('\nValidation Classification Report:\n', classification_report(y_val, y_val_pred))

# Test
y_test_proba = best_model.predict_proba(X_test)[:,1]
y_test_pred = best_model.predict(X_test)
print('Test AUC:', roc_auc_score(y_test, y_test_proba))
print('Test Precision:', precision_score(y_test, y_test_pred))
print('Test Recall:', recall_score(y_test, y_test_pred))
print('Test F1:', f1_score(y_test, y_test_pred))
print('\nTest Classification Report:\n', classification_report(y_test, y_test_pred))

# Confusion matrix plot
cm = confusion_matrix(y_test, y_test_pred)
fig, ax = plt.subplots(figsize=(5,4))
cax = ax.matshow(cm, cmap='Blues')
for (i, j), z in np.ndenumerate(cm):
    ax.text(j, i, str(z), ha='center', va='center')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Test)')
fig.colorbar(cax)
plt.show()


In [ ]:
# Save trained model and sample predictions; export the processed dataset subset
os.makedirs('artifacts', exist_ok=True)
joblib.dump(best_model, 'artifacts/best_model.joblib')
print('Saved model to artifacts/best_model.joblib')

# Save a sample of processed features to CSV for inspection (transform then inverse mapping not straightforward due to one-hot)
X_sample = X_test.copy().reset_index(drop=True)
X_sample['pred_proba'] = y_test_proba
X_sample['pred_label'] = y_test_pred
X_sample.to_csv('artifacts/sample_predictions.csv', index=False)
print('Saved sample predictions to artifacts/sample_predictions.csv')


## Deployment Readiness & Reflection

**Deployment plan (high level)**
1. Wrap `artifacts/best_model.joblib` in a FastAPI service with input validation and authentication.
2. Containerize with Docker and deploy to a secure environment (on-prem or private cloud).
3. Integrate via FHIR/HL7 connectors or EHR middleware; show risk scores in clinician dashboard.
4. Implement monitoring: rolling AUC, calibration, input distribution drift detection, and alerting.

**Ethical considerations**
- Ensure HIPAA compliance: encryption, access controls, BAAs.
- Audit model fairness across demographics and perform subgroup analyses.
- Keep clinicians in the loop; present explanations (e.g., top contributing features).

**Reflection**
- The synthetic dataset helps illustrate pipeline steps; real-world EHR data will require rigorous cleaning, de-identification, and governance.
